<a href="https://colab.research.google.com/github/alessandroantonucci/CredalCAT/blob/main/Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1HRnKVskd9rWaj-63e9mBE8KcDBXEdY6w?usp=sharing

In [26]:
import numpy as np
from math import log
from random import random

def f(x):
  return 1.0/(1.0+x)

def ld2p(l,d):
    return [1.-ll-dd/2.,1.-ll+dd/2.]

def updater(prior,cpts,questions,answers):
  posterior = np.array([prior,1.0-prior])
  for (q,a) in zip(questions,answers):
    if a == 0: # Q=0
      posterior[0] *= (1.0-cpts[q][0]) # P(S=1,Q=0) = P(S=1)*P(Q=0|S=1)
      posterior[1] *= (1.0-cpts[q][1]) # P(S=0,Q=0) = P(S=0)*P(Q=0|S=0)
    else: # Updating given a correct answer
      posterior[0] *= cpts[q][0] # P(S=1,Q=1) = P(S=1)*P(Q=1|S=1)
      posterior[1] *= cpts[q][1] # P(S=0,Q=1) = P(S=0)*P(Q=1|S=1)
  return posterior/sum(posterior) # P(S|Q=q) \prop P(S,q)

def c_updater(c_prior,c_cpts,questions,answers):
  lp0 = 1.0 # lP(S=0|...)
  up0 = 1.0 # uP(S=0|...)
  lp1 = 1.0 # lP(S=1|...)
  up1 = 1.0 # uP(S=1|...)
  for (q,a) in zip(questions,answers):
    if a == 0: # Q=0
      lp0 *= (1.0-c_cpts[q][0][0])/(1.0-c_cpts[q][1][1]) # upperP(Q=0|S=1)/lowerP(Q=0|S=0)
      up0 *= (1.0-c_cpts[q][0][1])/(1.0-c_cpts[q][1][0]) # lowerP(Q=0|S=1)/upperP(Q=0|S=0)
      lp1 *= (1.0-c_cpts[q][1][0])/(1.0-c_cpts[q][0][1]) # upperP(Q=0|S=0)/lowerP(Q=0|S=1)
      up1 *= (1.0-c_cpts[q][1][1])/(1.0-c_cpts[q][0][0]) # lowerP(Q=0|S=0)/upperP(Q=0|S=1)
    else: # Q=1
      lp0 *= c_cpts[q][0][0]/c_cpts[q][1][1] # upperP(Q=1|S=1)/lowerP(Q=1|S=0)
      up0 *= c_cpts[q][0][1]/c_cpts[q][1][0] # lowerP(Q=1|S=1)/upperP(Q=1|S=0)
      lp1 *= c_cpts[q][1][0]/c_cpts[q][0][1] # upperP(Q=1|S=0)/lowerP(Q=1|S=1)
      up1 *= c_cpts[q][1][1]/c_cpts[q][0][0] # lowerP(Q=1|S=0)/upperP(Q=1|S=1)
  gamma = [0.,0.]
  gamma[0] = c_prior[0]/(1.0-c_prior[0])
  gamma[1] = c_prior[1]/(1.0-c_prior[1])
  lp0 = min(f(gamma[0]*lp0),f(gamma[1]*lp0))
  up0 = max(f(gamma[0]*up0),f(gamma[1]*up0))
  lp1 = min(f(1.0/gamma[0]*lp1),f(1.0/gamma[1]*lp1))
  up1 = max(f(1.0/gamma[0]*up1),f(1.0/gamma[1]*up1))
  return [lp1,up1]
  
def entropy(x): # Entropy of [p,1-p]
    return -(x*log(x,2)+(1-x)*log(1-x,2))

def prob_question(p,c):
  # p(Q=1) = p(Q=1|S=1)P(S=1)+P(Q=1|S=0)P(S=0)
  return c[0]*p+c[1]*(1-p)

def expent_scores(p,c,q,a): # expected entropy
  scores = [0.0 for _ in range(len(c))]
  for i in range(len(c)):
    pS = updater(p,c,q,a) # [P(S=1|q=a),P(S=0|q=a)]
    p0 = updater(pS[0],c,q+[i],a+[0]) #[P(S=1|Q=0,q=a),P(S=0|Q=0,q=a)]
    p1 = updater(pS[0],c,q+[i],a+[1]) #[P(S=1|Q=1,q=a),P(S=0|Q=1,q=a)]
    pq = prob_question(pS[0],c[i]) # P(Q=1)
    scores[i] = pq*entropy(p1[0])+(1-pq)*entropy(p0[0]) #H[S|Q]
  return scores

p = 0.5 #P(S=1)

tables = [[0.,0.] for _ in range(4)]
tables[0] = [0.9, 0.4] #p(Q=1|S=1),P(Q=1|S=0)
tables[1] = [0.8, 0.5] 
tables[2] = [0.9, 0.5] 
tables[3] = [0.55, 0.25]

entropy_star = entropy(0.1) # threshold on the entropy
e = 1.0 # initialization
qst = [] # array with the integers denoting the templates of the ansers
ans = [] # array with the boolean integers denoting the answers to the questions in qst

# BAYESIAN
t = 0
for k in range(10):
#while e > entropy_star:
  posterior = updater(p,tables,qst,ans) #updating given previous answers
  picked_question = np.argmax(expent_scores(p,tables,qst,ans)) # picking the best question
  past = ""
  for (q,a) in zip(qst,ans):
    past += "Q"+str(q)+"="+str(a)+" "
  print("T=%d \t P(S=1|%s)=%2.4f" %(t,past,posterior[0])) # logging
  qst += [picked_question]
  given_answer = 0  
  if random()<prob_question(posterior[0],tables[picked_question]): #simulating the answer
    given_answer = 1
  ans += [given_answer]
  posterior = updater(p,tables,qst,ans) #updating given previous answers
  e = entropy(posterior[0])
  t += 1

# CREDAL
c_p = [0.4 , 0.55] # [lP(S=1),uP(S=1)]
c_tables = [ [[0.,0.],[0.,0.]] for _ in range(4)]
c_tables[0] = [[0.89, 0.91],[0.49,0.41]] #[[lp(Q=1|S=1),lp(Q=1|S=1)],[lP(Q=1|S=0),uP(Q=1|S=0)]
c_tables[1] = [[0.78, 0.81],[0.45,0.51]]
c_tables[2] = [[0.89, 0.91],[0.45,0.51]]
c_tables[3] = [[0.54, 0.57],[0.24,0.26]]


t = 0
qst = [] # array with the integers denoting the templates of the ansers
ans = [] # array with the boolean integers denoting the answers to the questions in qst

# CREDAL
t = 0
for k in range(10):
#while e > entropy_star:
  c_posterior = c_updater(c_p,c_tables,qst,ans) #updating given previous answers
  picked_question = np.argmax(expent_scores(p,tables,qst,ans)) # picking the best question
  past = ""
  for (q,a) in zip(qst,ans):
    past += "Q"+str(q)+"="+str(a)+" "
  print("T=%d \t P(S=1|%s)=[%2.4f,%2.4f]" %(t,past,c_posterior[0],c_posterior[1])) # logging
  qst += [picked_question]
  given_answer = 0  
  if random()<prob_question(posterior[0],tables[picked_question]): #simulating the answer
    given_answer = 1
  ans += [given_answer]
  posterior = updater(p,tables,qst,ans) #updating given previous answers
  e = entropy(posterior[0])
  t += 1

T=0 	 P(S=1|)=0.5000
T=1 	 P(S=1|Q3=0 )=0.3750
T=2 	 P(S=1|Q3=0 Q1=1 )=0.4898
T=3 	 P(S=1|Q3=0 Q1=1 Q3=0 )=0.3655
T=4 	 P(S=1|Q3=0 Q1=1 Q3=0 Q1=0 )=0.1873
T=5 	 P(S=1|Q3=0 Q1=1 Q3=0 Q1=0 Q3=1 )=0.3364
T=6 	 P(S=1|Q3=0 Q1=1 Q3=0 Q1=0 Q3=1 Q1=0 )=0.1686
T=7 	 P(S=1|Q3=0 Q1=1 Q3=0 Q1=0 Q3=1 Q1=0 Q3=0 )=0.1085
T=8 	 P(S=1|Q3=0 Q1=1 Q3=0 Q1=0 Q3=1 Q1=0 Q3=0 Q3=1 )=0.2111
T=9 	 P(S=1|Q3=0 Q1=1 Q3=0 Q1=0 Q3=1 Q1=0 Q3=0 Q3=1 Q3=0 )=0.1384
T=0 	 P(S=1|)=[0.4000,0.5500]
T=1 	 P(S=1|Q3=1 )=[0.6129,0.7174]
T=2 	 P(S=1|Q3=1 Q3=0 )=[0.4725,0.6121]
T=3 	 P(S=1|Q3=1 Q3=0 Q3=0 )=[0.3364,0.4952]
T=4 	 P(S=1|Q3=1 Q3=0 Q3=0 Q1=1 )=[0.4771,0.6000]
T=5 	 P(S=1|Q3=1 Q3=0 Q3=0 Q1=1 Q3=1 )=[0.6842,0.7570]
T=6 	 P(S=1|Q3=1 Q3=0 Q3=0 Q1=1 Q3=1 Q3=1 )=[0.8373,0.8662]
T=7 	 P(S=1|Q3=1 Q3=0 Q3=0 Q1=1 Q3=1 Q3=1 Q0=0 )=[0.4759,0.5468]
T=8 	 P(S=1|Q3=1 Q3=0 Q3=0 Q1=1 Q3=1 Q3=1 Q0=0 Q3=1 )=[0.6832,0.7148]
T=9 	 P(S=1|Q3=1 Q3=0 Q3=0 Q1=1 Q3=1 Q3=1 Q0=0 Q3=1 Q3=1 )=[0.8367,0.8388]


In [27]:
for x in range(10):
  if x>0:
    print("p=%2.4f \t H[p]=%2.4f"%(x/500.,entropy(x/500.)))

p=0.0020 	 H[p]=0.0208
p=0.0040 	 H[p]=0.0376
p=0.0060 	 H[p]=0.0529
p=0.0080 	 H[p]=0.0672
p=0.0100 	 H[p]=0.0808
p=0.0120 	 H[p]=0.0938
p=0.0140 	 H[p]=0.1063
p=0.0160 	 H[p]=0.1184
p=0.0180 	 H[p]=0.1301


In [28]:
p = 0.5 #P(S=1)
tables = [[0.,0.] for _ in range(2)]
tables[0] = [0.9, 0.3] #p(Q=1|S=1),P(Q=1|S=0)
tables[1] = [0.6, 0.4] 
print('P(S=1|Q0=0,Q1=0)',updater(p,tables,[0,1],[0,0]))
print('P(S=1|Q0=0)',updater(p,tables,[0],[0]))
print('P(S=1|Q0=0,Q1=1)',updater(p,tables,[0,1],[0,1]))
print('P(S=1|Q1=0)',updater(p,tables,[1],[0]))
print('P(S=1|Q1=1)',updater(p,tables,[1],[1]))
print('P(S=1|Q0=1,Q1=0)',updater(p,tables,[0,1],[1,0]))
print('P(S=1|Q0=1)',updater(p,tables,[0],[1]))
print('P(S=1|Q0=1,Q1=1)',updater(p,tables,[0,1],[1,1]))


# CREDAL
c_p = [0.45 , 0.55] # [lP(S=1),uP(S=1)]
c_tables = [ [[0.,0.],[0.,0.]] for _ in range(2)]
c_tables[0] = [[0.85, 0.95],[0.25,0.35]] #[[lp(Q=1|S=1),lp(Q=1|S=1)],[lP(Q=1|S=0),uP(Q=1|S=0)]
c_tables[1] = [[0.55, 0.65],[0.35,0.45]]

print('P(S=1|Q0=0,Q1=0)',c_updater(c_p,c_tables,[0,1],[0,0]))
print('P(S=1|Q0=0)',c_updater(c_p,c_tables,[0],[0]))
print('P(S=1|Q0=0,Q1=1)',c_updater(c_p,c_tables,[0,1],[0,1]))
print('P(S=1|Q1=0)',c_updater(c_p,c_tables,[1],[0]))
print('P(S=1|Q1=1)',c_updater(c_p,c_tables,[1],[1]))
print('P(S=1|Q0=1,Q1=0)',c_updater(c_p,c_tables,[0,1],[1,0]))
print('P(S=1|Q0=1)',c_updater(c_p,c_tables,[0],[1]))
print('P(S=1|Q0=1,Q1=1)',c_updater(c_p,c_tables,[0,1],[1,1]))


#  t += 1

P(S=1|Q0=0,Q1=0) [0.08695652 0.91304348]
P(S=1|Q0=0) [0.125 0.875]
P(S=1|Q0=0,Q1=1) [0.17647059 0.82352941]
P(S=1|Q1=0) [0.4 0.6]
P(S=1|Q1=1) [0.6 0.4]
P(S=1|Q0=1,Q1=0) [0.66666667 0.33333333]
P(S=1|Q0=1) [0.75 0.25]
P(S=1|Q0=1,Q1=1) [0.81818182 0.18181818]
P(S=1|Q0=0,Q1=0) [0.02853260869565219, 0.18749999999999997]
P(S=1|Q0=0) [0.051724137931034524, 0.22000000000000003]
P(S=1|Q0=0,Q1=1) [0.09198113207547177, 0.25635593220338987]
P(S=1|Q1=0) [0.3058252427184466, 0.5]
P(S=1|Q1=1) [0.6030927835051547, 0.5990099009900991]
P(S=1|Q0=1,Q1=0) [0.6260460251046025, 0.7083333333333334]
P(S=1|Q0=1) [0.7566371681415929, 0.748]
P(S=1|Q0=1,Q1=1) [0.8523773006134969, 0.7839176829268293]


$$P(s|q)=\frac{P(s)P(q|s)}{P(s)P(q|s) + P(\neg s)P(q|\neg s)}=\frac{P(s)}{P(s) + (1-P(s))\frac{P(q|\neg s)}{P(q|s)}}$$

$$\underline{P}(s|q)=\min_{P(S)} \frac{P(s)}{P(s)+(1-P(s))\max_{P(Q|S)} \frac{P(q|\neg s)}{P(q|s)}}$$

$$\underline{P}(s|q)=\min_{P(S)} \frac{P(s)}{P(s)+(1-P(s)) \frac{\overline{P}(q|\neg s)}{\underline{P}(q|s)}}$$
